# Creating the tardis_example database

This notebook provides a quick introduction in how to use Carsus.

## Prerequisites:
- Install carsus (preferably as a git repository)
- Install ChiantiPy (from source or pip, both versions work)
- Install chiantipy and set the environmentvariable `XUVTOP = /path/to/chianti/root`
- We highly recommend installing the `carsus-db` repository as it includes the files of several sources that cannot easily be downloaded.

## Imports and initialization:

Import all the essential ingesters and initialize the session.

In [1]:
import os
from carsus import init_db
from carsus.io.nist import (
        NISTWeightsCompIngester,
        NISTIonizationEnergiesIngester
        )
from carsus.io.kurucz import GFALLIngester
from carsus.io.chianti_ import ChiantiIngester
from carsus.io.zeta import KnoxLongZetaIngester
from carsus.io.output import AtomData


DB_ROOT = '/media/data/projects/tardis/carsus-db/'
session = init_db('sqlite:////tmp/carsus_example.db')
#session = init_db('sqlite:///' + os.path.join(DB_ROOT, 'carsus_example.db'))
storename = os.path.join(DB_ROOT, 'kurucz_chianti_H_He.h5')
storename = '/tmp/carsus_example.h5'
zeta_fname = os.path.join(DB_ROOT, 'zeta/knox_long_recombination_zeta.dat')
gfall_fname = os.path.join(DB_ROOT, 'gfall/gfall_old.dat')

 found PyQt4 widgets
 using PyQt4 widgets
Initializing the database at sqlite:////tmp/carsus_example.db
Ingesting basic atomic data


## Ingesting data


### Weights

In [2]:
weightscomp_ingester = NISTWeightsCompIngester(session)
weightscomp_ingester.ingest()
session.commit()

Ingesting atomic weights from nist


Now we have all the weights ingested.

### Ionization Energies

In [3]:
ioniz_energies_ingester = NISTIonizationEnergiesIngester(
            session,
            spectra="h-zn"
            )
ioniz_energies_ingester.ingest(
            ionization_energies=True,
            ground_levels=True
            )
session.commit()


/media/data/projects/tardis/carsus/carsus/io/nist/ionization.py:88: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  usecols=range(5), names=column_names, skiprows=3, skipfooter=1)


Ingesting ionization energies from nist-asd
Ingesting ground levels from nist-asd


### Ingesting kurucz levels and lines

In [4]:
gfall_ingester = GFALLIngester(session, gfall_fname, ions='H-Zn')
gfall_ingester.ingest(levels=True, lines=True)
session.commit()

Ingesting levels from ku_latest
Ingesting levels for He 0
Ingesting levels for He 1
Ingesting levels for Li 0
Ingesting levels for Li 1
Ingesting levels for Be 0
Ingesting levels for Be 1
Ingesting levels for Be 2
Ingesting levels for B 0
Ingesting levels for B 1
Ingesting levels for B 2
Ingesting levels for B 3
Ingesting levels for C 0
Ingesting levels for C 1
Ingesting levels for C 2
Ingesting levels for C 3
Ingesting levels for N 0
Ingesting levels for N 1
Ingesting levels for N 2
Ingesting levels for N 3
Ingesting levels for N 4
Ingesting levels for N 5
Ingesting levels for O 0
Ingesting levels for O 1
Ingesting levels for O 2
Ingesting levels for O 3
Ingesting levels for O 4
Ingesting levels for O 5
Ingesting levels for F 0
Ingesting levels for F 1
Ingesting levels for F 2
Ingesting levels for F 3
Ingesting levels for F 4
Ingesting levels for F 5
Ingesting levels for Ne 0
Ingesting levels for Ne 1
Ingesting levels for Ne 2
Ingesting levels for Ne 3
Ingesting levels for Ne 4
Ingest

### Ingesting chianti levels and lines for H and He

In [5]:
chianti_ingester = ChiantiIngester(session, ions='h-he')
chianti_ingester.ingest(levels=True, lines=True, collisions=False)
session.commit()

Ingesting levels from chianti_v8.0.6
Ingesting levels for H 0
Ingesting levels for He 0
Ingesting levels for He 1
Ingesting lines from chianti_v8.0.6
Ingesting lines for H 0
Ingesting lines for He 0
Ingesting lines for He 1


### Ingesting Zeta data

In [6]:
zeta_ingester = KnoxLongZetaIngester(session, zeta_fname)
zeta_ingester.ingest()
session.commit()

## Create a tardis readable HDFStore

As we did in the Quickstart, we first create an `AtomData` instance. For this database we choose kurucz as the primary `DataSource` with only hydrogen and helium coming from chianti. We also specify `chianti_short_name` which contains the version of the chianti database that was ingested.

In [7]:
ad = AtomData(
        session,
        selected_atoms='H-Zn',
        chianti_ions="H; He",
        chianti_short_name='chianti_v8.0.6'
        )



### Writing the output

The database we want to create contains all data, except for the collision data.

In [8]:
ad.to_hdf(
        storename,
        store_atom_masses=True,
        store_ionization_energies=True,
        store_levels=True,
        store_lines=True,
        store_macro_atom=True,
        store_collisions=False,
        store_zeta_data=True
        )

/media/data/projects/tardis/carsus/carsus/io/output/tardis_.py:562: RuntimeWarning: divide by zero encountered in log10
  lines['loggf'] = np.log10(lines['gf'])


Signing AtomData: 
MD5: 3c7174aa939d1c9dac4e41afd7168796
UUID1: 4f8355067e6a11e78bcb507b9d2c8db1


Comparison to old kurucz: 7 lines missing

You are done! Now you can use the created HDFStore to run TARDIS simulations.